In [33]:
import cv2
import numpy as np
import pyvista as pv
from pyvista import examples
import tifffile as tiff

# Load disparity map and corresponding RGB image
disparity_map = tiff.imread('../data/kramlich-subset/depth/device_741df78f3cf22b9e_camera1_1722960518099.tiff')
rgb_image = cv2.imread('../data/kramlich-subset/rgb/device_741df78f3cf22b9e_camera1_1722960518099.jpg')

# Handle potential nan or inf values in the disparity map
disparity_map[disparity_map < 0] = 0.0

# Apply a threshold to avoid near-zero disparities
disparity_threshold = 2.0  # Set an appropriate threshold (adjust based on your system)
disparity_map[disparity_map < disparity_threshold] = 0.0

# Camera parameters (adjust as per your setup)
focal_length = 100.0  # Example focal length in pixels
baseline = 1.0  # Baseline in meters
cx, cy = rgb_image.shape[1] // 2, rgb_image.shape[0] // 2  # Assume image center is the principal point
fx, fy = focal_length, focal_length

# Construct the Q matrix
Q = np.array([[1, 0, 0, -cx],
              [0, 1, 0, -cy],
              [0, 0, 0, focal_length],
              [0, 0, -1.0 / baseline, 0]], dtype=np.float32)

# Reproject points into 3D space using the Q matrix
points_3d = cv2.reprojectImageTo3D(disparity_map, Q)

# Mask out invalid points where disparity was zero or still invalid
mask = (disparity_map >= disparity_threshold) & (disparity_map < np.inf)

# Extract valid 3D points and corresponding RGB colors
valid_points = points_3d[mask]
valid_colors = rgb_image[mask].astype(np.float32) / 255.0  # Normalize colors to [0, 1]

# Create a PyVista point cloud object
point_cloud = pv.PolyData(valid_points)

# Add RGB colors to the point cloud
point_cloud['RGB'] = valid_colors


In [35]:
import numpy as np

# Assuming valid_points contains your (X, Y, Z) data and valid_colors contains (R, G, B)
valid_points = np.array(valid_points)

# Option 1: Center the points around the mean (without scaling)
centered_points = valid_points - np.mean(valid_points, axis=0)

# Option 2: If you want to scale (optional), apply a fixed scaling factor
# This is not necessary but can help if the points are too far apart or too close.
scaling_factor = 1  # You can set this to another value to adjust the point cloud's scale
scaled_points = centered_points * scaling_factor

# Convert the RGB colors to the range [0, 255] for saving in PLY format
valid_colors_255 = (valid_colors * 255).astype(np.uint8)  # Convert colors back to 255 scale

# Combine the points and colors into one array for saving
points_with_colors = np.hstack([scaled_points, valid_colors_255])

# Prepare the PLY file header
ply_header = '''ply
format ascii 1.0
element vertex {0}
property float x
property float y
property float z
property uchar red
property uchar green
property uchar blue
end_header
'''.format(len(points_with_colors))

# Write the point cloud to a PLY file
with open('point_cloud.ply', 'w') as f:
    f.write(ply_header)
    np.savetxt(f, points_with_colors, fmt='%f %f %f %d %d %d')


In [4]:
import numpy as np
from scipy.spatial import KDTree
from numba import jit

@jit(nopython=True)
def find_closest_points(source_points, target_points):
    """Find the closest points in target for each point in source."""
    closest_points = np.zeros_like(source_points)
    distances = np.zeros(source_points.shape[0])

    for i in range(source_points.shape[0]):
        diff = target_points - source_points[i]
        dist_sq = np.sum(diff ** 2, axis=1)
        closest_idx = np.argmin(dist_sq)
        closest_points[i] = target_points[closest_idx]
        distances[i] = np.sqrt(dist_sq[closest_idx])

    return closest_points, distances

@jit(nopython=True)
def calculate_mean(points):
    """Manually calculate the mean of a set of points (Numba-compatible)."""
    return np.sum(points, axis=0) / points.shape[0]

@jit(nopython=True)
def best_fit_transform(A, B):
    """Calculates the best-fit transform that maps points A to points B."""
    assert A.shape == B.shape

    # Manually compute centroids
    centroid_A = calculate_mean(A)
    centroid_B = calculate_mean(B)

    # Center the points
    AA = A - centroid_A
    BB = B - centroid_B

    # Compute the covariance matrix
    H = np.dot(AA.T, BB)

    # Compute the Singular Value Decomposition (SVD)
    U, S, Vt = np.linalg.svd(H)

    # Compute rotation
    R = np.dot(Vt.T, U.T)

    # Ensure a proper rotation matrix (det(R) should be 1)
    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = np.dot(Vt.T, U.T)

    # Compute translation
    t = centroid_B.T - np.dot(R, centroid_A.T)

    return R, t

@jit(nopython=True)
def icp(source_points, target_points, max_iterations=100, tolerance=1e-6):
    """Perform ICP to align source_points with target_points."""
    prev_error = float('inf')

    # Initial alignment (assume identity matrix)
    source_transformed = np.copy(source_points)

    for i in range(max_iterations):
        # Find closest points in target for each point in source
        closest_points, distances = find_closest_points(source_transformed, target_points)

        # Compute the best-fit transform
        R, t = best_fit_transform(source_transformed, closest_points)

        # Apply the transformation
        source_transformed = np.dot(source_transformed, R.T) + t

        # Check for convergence
        mean_error = np.sum(distances) / distances.shape[0]
        if abs(prev_error - mean_error) < tolerance:
            break
        prev_error = mean_error

    return source_transformed, R, t


In [5]:
import cv2
import numpy as np
import pyvista as pv
from pyvista import examples
import tifffile as tiff


# Load disparity map and corresponding RGB image
disparity_map_1 = tiff.imread('../data/kramlich-subset/depth/device_741df78f3cf22b9e_camera1_1722960518099.tiff')
rgb_image_1 = cv2.imread('../data/kramlich-subset/rgb/device_741df78f3cf22b9e_camera1_1722960518099.jpg')
disparity_map_2 = tiff.imread('../data/kramlich-subset/depth/device_741df78f3cf22b9e_camera1_1722960517996.tiff')
rgb_image_2 = cv2.imread('../data/kramlich-subset/rgb/device_741df78f3cf22b9e_camera1_1722960517996.jpg')

# Convert the images from BGR to RGB
rgb_image_1 = cv2.cvtColor(rgb_image_1, cv2.COLOR_BGR2RGB)
rgb_image_2 = cv2.cvtColor(rgb_image_2, cv2.COLOR_BGR2RGB)

# Set parameters for your stereo setup (adjust based on your camera)
focal_length = 100.0  # Focal length in pixels
baseline = 1.0  # Baseline in meters

# Process both disparity maps and images into 3D point clouds
def disparity_to_point_cloud(disparity_map, rgb_image, threshold=2.0):
    """Convert a disparity map and corresponding RGB image into a point cloud."""
    # Handle any negative or invalid values in the disparity map
    disparity_map[disparity_map < 0] = 0.0
    disparity_map[disparity_map < threshold] = 0.0

    # Image and camera parameters (adjust as necessary)
    cx, cy = rgb_image.shape[1] // 2, rgb_image.shape[0] // 2  # Assume the image center is the principal point
    fx, fy = focal_length, focal_length

    # Construct the Q matrix for reprojection to 3D
    Q = np.array([[1, 0, 0, -cx],
                  [0, 1, 0, -cy],
                  [0, 0, 0, focal_length],
                  [0, 0, -1.0 / baseline, 0]], dtype=np.float32)

    # Reproject to 3D
    points_3d = cv2.reprojectImageTo3D(disparity_map, Q)

    # Mask invalid points (where disparity is zero or invalid)
    mask = (disparity_map >= threshold) & (disparity_map < np.inf)

    # Extract valid points and their corresponding colors
    valid_points = points_3d[mask]
    valid_colors = rgb_image[mask].astype(np.float32) / 255.0  # Normalize colors to [0, 1]

    return valid_points, valid_colors

# Get point clouds for both maps
valid_points_1, valid_colors_1 = disparity_to_point_cloud(disparity_map_1, rgb_image_1)
valid_points_2, valid_colors_2 = disparity_to_point_cloud(disparity_map_2, rgb_image_2)


In [6]:
# Apply ICP to align point clouds
aligned_points_2, R, t = icp(valid_points_2, valid_points_1)

# Merge the aligned points with the first point cloud
merged_points = np.vstack((valid_points_1, aligned_points_2))
merged_colors = np.vstack((valid_colors_1, valid_colors_2))

# Create a PyVista point cloud object for the merged point cloud
merged_point_cloud = pv.PolyData(merged_points)
merged_point_cloud['RGB'] = merged_colors

# Visualize the combined point cloud
plotter = pv.Plotter()
plotter.add_points(merged_point_cloud, scalars='RGB', rgb=True, point_size=5)
plotter.show()
